Carga de archivo
----

Se lee el archivo con listas de películas y se almacena en mapas. El formato del archivo es:
````
id_lista1, movie1, movie2,...
id_lista2, ...
....
````

In [37]:
import csv 
lsname = "asmr"
filename = "asmr.txt"

movie2list = dict()
list2movie = dict()

mov2index = dict();
all_movies = [] # con repetición
all_movies2 = []; # sin repetición
idm=0


with open(filename) as tsv:
    for mlist in csv.reader(tsv, dialect="excel-tab"):
        while '' in mlist: mlist.remove('')
        id_list = mlist[0]
        list2movie[id_list] = set(mlist[1:])
        for movie in mlist[1:]:
            if movie in movie2list:
                movie2list[movie].add(id_list)
            else: 
                movie2list[movie] = set([id_list])
                
            if movie not in all_movies2:
                mov2index[movie]=idm
                all_movies2.append(movie)
                idm += 1
            all_movies.append(movie)
                
lssize = len(list2movie)
print("Total de peliculas:",len(mov2index))
print("Total de peliculas (contando repetidas):",len(all_movies))
print("Total de listas:",lssize)

Total de peliculas: 1248
Total de peliculas (contando repetidas): 2967
Total de listas: 225


In [16]:
def get_movies(idList):      
    return list2movie[idList]
print(get_movies(lsname+"1001"))

{'moonlight-2016', 'uncle-boonmee-who-can-recall-his-past-lives', 'the-tree-of-life-2011', 'the-diving-bell-and-the-butterfly', 'whiplash-2014', 'where-the-wild-things-are', 'birdman-or-the-unexpected-virtue-of-ignorance', 'mulholland-drive', 'uncut-gems', 'roma-2018'}


In [17]:
def get_lists(idMovie):      
    return movie2list[idMovie]


print(get_lists(list(get_movies(lsname+"1001"))[0]))

{'asmr1141', 'asmr1172', 'asmr1111', 'asmr1080', 'asmr1075', 'asmr1092', 'asmr1108', 'asmr1052', 'asmr1138', 'asmr1027', 'asmr1048', 'asmr1043', 'asmr1081', 'asmr1097', 'asmr1079', 'asmr1001', 'asmr1056', 'asmr1102', 'asmr1091', 'asmr1206', 'asmr1041', 'asmr1040', 'asmr1209', 'asmr1087'}


mov2vec
---

Esta función transforma una lista de películas `mlist` en un vector `[0 0 0 0 X .... X .. X .. 0]`.
Cada casilla del vector represnta a una película, y las `X=1/len(mlist)` corresponden a las películas de la lista.

In [18]:
import random
import numpy as np
def mov2vec(mlist, mov2index):
    size = len (mov2index)
    vec = np.zeros(size)
    for m in mlist:
        vec[mov2index[m]] = 1.0/len(mlist)
    return vec

Generación de Datos de entrenamiento
----

Cada dato de entrenamiento se genera de la siguiente forma:

1. Se selecciona una lista aleatoria `L` de la primera mitad de listas (`lssize/2`).
2. La entrada (`data_x`) corresponde a una de entre 2 y 5 películas pertenecienta a `L`.
3. La salida (`data_y`) coresponde a la lista de películas completa `L`

Notar que las listas son vectorizadas usando la función `mov2vec`

In [19]:
n_data = 10000
data_x = np.zeros((n_data, len(mov2index))); data_y = np.zeros((n_data, len(mov2index)));
for i in range(0,n_data):
    j = random.randint(1001,1000+int(lssize/2))
    L = list(get_movies(lsname+str(j)))
    sz = len(L)
    cut = random.randint(5,10)
    random.shuffle(L)
    l1 = L[:cut]
    data_x[i] = mov2vec(l1, mov2index)
    data_y[i] = mov2vec(L, mov2index)

Modelo de red neuronal
---

Se crea la red neuronal densa. Tanto la entrada como la salida es un vector de largo igual a la cantidad total de películas.

In [20]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

N = len (mov2index)
def baseline_model():
    # define the keras model
    model = Sequential()
    model.add(Dense(20, input_dim=N, activation='sigmoid'))  # "attributes" layer
    #model.add(Dense(50, activation='relu'))  # "attributes" layer
    model.add(Dense(N))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model

Ajuste de la red
---

La red se entrena usando los datos creados más arriba

In [21]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=500, verbose=False)
estimator.fit(data_x, data_y)
#kfold = KFold(n_splits=10)
#results = cross_val_score(estimator, data_x, data_y, cv=kfold)
#print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Recommend (ANN)
----

La función `ann_recommend` usa la red entrenada para realizar una recomendación.

La función retorna una lista de `n` películas que recomienda a partir de una lista de entrada `F`. Las películas recomendadas no están en el conjunto `F`.

Además se puede incluir una lista `seen` con las películas vistas por el usuario para que no aparezcan entre las recomendaciones.


In [22]:
def ann_recommend(F, n=10, seen=[], verbose=True):
    data_x = mov2vec(F, mov2index)
    
    data_y = estimator.predict([[data_x]])[0]
    
    max_indexes = (-data_y).argsort() #índices ordenados
    recommendations = set()
    for i in max_indexes:
        if all_movies2[i] not in F and all_movies2[i] not in seen:
            if(verbose): print(data_y[i], all_movies2[i])
            recommendations.add (all_movies2[i])
            if len(recommendations) == n: break  
    return recommendations

En este ejemplo se muestran las recomendaciones a partir de una lista dada. Adjunto a cada recomendación, el valor asociado a cada película por la red (recuerda que la red retorna un vector en donde cada casilla corresponde a una película, las películas recomendadas son las que maximizan el valor).

> Cabe mencionar que utilicé el recomendador con esta lista F para que me recomendara películas y me han parecido bastante buenas las que he visto hasta ahora.

In [23]:
F = ['little-forest','columbus-2017','kikis-delivery-service','the-assassination-of-jesse-james-by-the-coward-robert-ford']
ann_recommend(F,10,verbose=True); 

0.021584928 2001-a-space-odyssey
0.017897815 portrait-of-a-lady-on-fire
0.01672056 phantom-thread
0.015966322 paterson
0.015290158 moonlight-2016
0.013575028 roma-2018
0.0132684065 her
0.0118889045 spirited-away
0.011280146 my-neighbor-totoro
0.011270956 the-tree-of-life-2011


### Ejemplo

En este ejemplo se ve el comportamiento del recomendador usando una de las listas de entrenamiento.

In [25]:
m = list(get_movies(lsname+"1010"))
random.shuffle(m)
l1 = m[:3]
l2 = m[3:]

data = mov2vec(l1, mov2index)

print("Lista original:",m)
print("Lista de entrada:", l1)
rec=ann_recommend(l1,5,verbose=False)
print("Películas recomendadas:",list(rec))
print("Aciertos:",len(rec.intersection(m)),"/5")

Lista original: ['boy-the-world', 'in-the-crosswind', 'the-fall', 'girls-lost', 'melancholia', 'bait-2019', 'a-ghost-story-2017', 'night-is-short-walk-on-girl', 'upstream-color', 'dead-slow-ahead']
Lista de entrada: ['boy-the-world', 'in-the-crosswind', 'the-fall']
Películas recomendadas: ['moonlight-2016', '2001-a-space-odyssey', 'portrait-of-a-lady-on-fire', 'phantom-thread', 'paterson']
Aciertos: 0 /5


En este ejemplo se ve el comportamiento del recomendador usando una de las listas **no usadas para entrenar**.

In [26]:
m = list(get_movies(lsname+"1172"))
random.shuffle(m)
l1 = m[:4]
l2 = m[4:]

data = mov2vec(l1, mov2index)
print("Lista original:",m)
print("Lista de entrada:", l1)

rec=ann_recommend(l1,10,verbose=False)
print("Películas recomendadas:",list(rec))
print("Aciertos:",len(rec.intersection(m)),"/10")

Lista original: ['portrait-of-a-lady-on-fire', 'stalker', 'the-florida-project', '2001-a-space-odyssey', 'blade-runner-2049', 'la-la-land', 'personal-shopper', 'the-lighthouse-2019', 'cold-war-2018', 'the-tree-of-life-2011', 'never-rarely-sometimes-always', 'roma-2018', 'columbus-2017', 'the-witch-2015', 'waves-2019', 'your-name', 'her', 'at-eternitys-gate', 'high-life-2018', 'arrival-2016', 'clementine-2019', 'moonlight-2016', 'ocean-waves', 'enter-the-void', 'gods-own-country-2017', 'a-ghost-story-2017', 'heartstone']
Lista de entrada: ['portrait-of-a-lady-on-fire', 'stalker', 'the-florida-project', '2001-a-space-odyssey']
Películas recomendadas: ['moonlight-2016', 'the-lighthouse-2019', 'uncle-boonmee-who-can-recall-his-past-lives', 'blade-runner', 'columbus-2017', 'phantom-thread', 'her', 'roma-2018', 'paterson', 'eraserhead']
Aciertos: 5 /10


### User-based Collaborative Filtering

Paper: [Recommender system techniques applied to Netflix movie data (2018)](https://science.vu.nl/en/Images/werkstuk-postmus_tcm296-877824.pdf)

Encontrar k usuarios más cercanos a usuario u, es decir, usuario que maximiza:

![image](https://i.imgur.com/ud1awnH.png)

La nota predicha para  las películas del usuario se obtiene promediando las notas de los usuarios más similares.

In [36]:
def sim_cosine(setA, setB):
    return len(setA.intersection(setB))/len(setA.union(setB))

1001

**Retorna índices de las listas mas cercanas a set_user**

In [27]:
def knn(set_user, k=5, ini=1001, end=1101):
    sim = np.zeros(end-ini+1)
    for i in range(ini,end+1):
        sim[i-ini] = (sim_cosine(set_user, get_movies(lsname+str(i))))
    return (-sim).argsort()[0:k]+ini

Ejemplo

In [44]:
F = set(["mulholland-drive","the-tree-of-life-2011","roma-2018","uncut-gems"])
sets = knn(F)
for s in sets:
    print(str(lsname+str(s)),":", sim_cosine(F, get_movies(lsname+str(s))))

asmr1001 : 0.4
asmr1060 : 0.16666666666666666
asmr1053 : 0.09090909090909091
asmr1048 : 0.09090909090909091
asmr1018 : 0.08333333333333333


In [45]:
def cf_recommend(F, n=10, seen=[], verbose=False):
    F=set(F)
    ids = knn(F,10,ini=1001,end=1000+int(lssize/2))
    m_counts = dict()
    for id in ids:
        for m in get_movies(lsname+str(id)):
            if m in m_counts: m_counts[m] +=1
            else: m_counts[m] = 1
                
    sorted_map = dict(sorted(m_counts.items(), key=lambda item: -item[1]))
    
    recommendations = set()
    for m in sorted_map:
        if m not in F and m not in seen:
            recommendations.add (m)
            if len(recommendations) == n: break  
    return recommendations


Validación
----

Para validar que el asunto haga cosas con sentido, se usa la siguiente función de validación.

La función `validate` realiza 10 pruebas aleatorias para cada lista que consisten en lo siguiente:

1. Se seleccionan `nsample` películas de la lista.
2. A partir de estas películas se recomiendan `nrec`
3. Se calcula la cantidad de aciertos entre las películas recomendadas y las que no fueron sampleadas de la lista.

Para comparar, también se genera una recomendación aleatoria que se obtiene seleccionando películas aleatorias de una **lista de todas las películas con repetición** (para que así las más repetidas tengan mayor probabilidad).
Luego se calcula la cantidad de aciertos de la misma manera que usando el recomendador "inteligente".




In [46]:
import random
random.seed(1)
def validate(list_name, nsample=3, nrec=5):
    ann_tot_success = 0
    cf_tot_success = 0
    rand_tot_success = 0
    tot_size = 0
    
    L = get_movies(list_name) #se obtienen las pelis de la lista
    for m in L: all_movies.remove(m)
    if verbose: print("movies:", L)

    #se eliminan de M las películas que solo aprecen en esta lista
    L2 = [x for x in L if x in all_movies]
    
    #Se considera el caso sólo si existen al menos nsample+2 películas en M
    if len(L2)>=nsample+2:
        hits_ann = 0
        hits_cf = 0
        for i in range(10):
            # recommendation using ANN
            source_movies = random.sample(L2,nsample) #seleccionamos nsample pelis al azar
            if verbose: print("source_movies:",source_movies)

            recommendations = ann_recommend(source_movies, n=nrec, verbose=False) 
            MM = [x for x in L2 if x not in source_movies]
            hits_ann += len(recommendations.intersection(MM))
            ann_tot_success += len(recommendations.intersection(MM))/float(len(L2)-nsample)
            tot_size += len(recommendations)
            if verbose: print("ann_rec:",recommendations)
                
            # recommendation using collaborative filtering
            source_movies = random.sample(L2,nsample) #seleccionamos nsample pelis al azar
            if verbose: print("source_movies:",source_movies)

            recommendations = cf_recommend(source_movies, n=nrec, verbose=False) 
            MM = [x for x in L2 if x not in source_movies]
            hits_cf += len(recommendations.intersection(MM))
            cf_tot_success += len(recommendations.intersection(MM))/float(len(L2)-nsample)
            tot_size += len(recommendations)
            if verbose: print("ann_rec:",recommendations)

            # random recommendation (conjunto con repetición)
            recommendations = set()
            #random.sample(all_movies,nrec)
            while len(recommendations) < nrec:
                recommendations=recommendations.union(set(random.sample(all_movies,nrec-len(recommendations))))
            rand_tot_success += len(recommendations.intersection(MM))/float(len(L2)-nsample)
            if verbose: print("rand_rec",recommendations)
        
        print(list_name +"\t"+ "%.1f" %(rand_tot_success*10) +"\t"+ "%.1f" %(ann_tot_success*10) +"\t" \
              + str(hits_ann/10.0) + "/" + str(len(L2)-nsample) +"\t"+ "%.1f" %(cf_tot_success*10) +"\t" \
              + str(hits_cf/10.0) + "/" + str(len(L2)-nsample) +"\t" + str(len(L2)) )

        for m in L: all_movies.append(m)
        return rand_tot_success/10.0,ann_tot_success/10.0,cf_tot_success/10.0
    
    for m in L: all_movies.append(m)
    return None, None, None

Se ejecuta el experimento de validación con las listas que **no se usaron par el entrenamiento**.

In [47]:
verbose=False
from IPython.display import display, Markdown, Latex
display(Markdown('rand: %acierto de recomendador aleatorio'))
display(Markdown('ann: %acierto de recomendador ANN'))
display(Markdown('hits: promedio de aciertos de las 10 películas recomendadas'))
display(Markdown('lenL: largo de la lista'))

print("lista \t\trand \tann \thits \tcf \thits \tlenL")
tot_rnd_success = 0.0; tot_ann_success=0; tot_cf_success=0; count=0
for i in range(int(lssize/2)+1,lssize):
    rnd_success, ann_success, cf_success = validate(lsname+str(1000+i), nsample=5, nrec=10)
    if rnd_success != None:
        tot_rnd_success += rnd_success
        tot_ann_success += ann_success
        tot_cf_success += cf_success
        count +=1
print("% acierto del recomendador ANN:",  "%.1f" %(tot_ann_success/count*100))
print("% acierto recomendador collaborative filtering:", "%.1f" %(tot_cf_success/count*100))
print("% acierto recomendador aleatorio:", "%.1f" %(tot_rnd_success/count*100))

rand: %acierto de recomendador aleatorio

ann: %acierto de recomendador ANN

hits: promedio de aciertos de las 10 películas recomendadas

lenL: largo de la lista

lista 		rand 	ann 	hits 	cf 	hits 	lenL
asmr1113	2.0	0.0	0.0/5	16.0	0.8/5	10
asmr1114	6.7	40.0	1.2/3	36.7	1.1/3	8
asmr1115	2.0	0.0	0.0/5	6.0	0.3/5	10
asmr1118	10.0	17.5	0.7/4	35.0	1.4/4	9
asmr1119	10.0	25.0	1.0/4	37.5	1.5/4	9
asmr1120	1.4	0.0	0.0/22	5.0	1.1/22	27
asmr1121	6.9	7.7	1.0/13	9.2	1.2/13	18
asmr1123	12.0	26.0	1.3/5	26.0	1.3/5	10
asmr1124	2.7	4.6	1.2/26	5.4	1.4/26	31
asmr1125	0.0	0.0	0.0/3	23.3	0.7/3	8
asmr1126	5.0	25.0	0.5/2	15.0	0.3/2	7
asmr1127	2.8	7.6	1.9/25	8.0	2.0/25	30
asmr1128	2.0	12.0	0.6/5	4.0	0.2/5	10
asmr1130	0.0	26.0	1.3/5	30.0	1.5/5	10
asmr1131	0.0	0.0	0.0/4	10.0	0.4/4	9
asmr1132	3.3	0.0	0.0/3	0.0	0.0/3	8
asmr1134	5.0	5.0	0.1/2	35.0	0.7/2	7
asmr1135	2.0	0.0	0.0/10	0.0	0.0/10	15
asmr1136	6.7	26.7	0.8/3	26.7	0.8/3	8
asmr1137	7.5	17.5	0.7/4	7.5	0.3/4	9
asmr1138	0.0	7.5	0.3/4	2.5	0.1/4	9
asmr1139	2.0	9.0	0.9/10	8.0	0.8/10	15
asmr1141	2.1	12.1	2.9/24	7.9	1.9/24	29
asmr1143	3.3	23.3	0.7/3	53.3	1.6/3	8
asmr1144	5.6	8.9	0.8/9	10.0	0.9/9	14
asmr1146	0.0	10.0	0.2/2	0.0	0.0